# Model Selection and Regularization (Seleção de Modelos e Regularização)

Prof. Daniel de Abreu Pereira Uhr

### Conteúdo
* Introdução
* Cross-Validation (Validação Cruzada)
  * The Validation Set Approach (A Abordagem do Conjunto de Validação)
  * Leave-One-Out Cross-Validation (Validação Cruzada Leave-One-Out)
  * k-Fold Cross-Validation (Validação Cruzada k-Fold)
* The Bootstrap

### Referências

* [An Introduction to Statistical Learning](https://www.statlearning.com/) (ISL) by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani
  * Capítulo 5***
* [The Elements of Statistical Learning](https://hastie.su.domains/ElemStatLearn/) (ESL) by Trevor Hastie, Robert Tibshirani and Jerome Friedman : 
  * Capítulo 7

***Disclaimer:*** *O material apresentado aqui é uma adaptação do material de aula do Prof. Daniel de Abreu Pereira Uhr, e não deve ser utilizado para fins comerciais. O material é disponibilizado para fins educacionais e de pesquisa, e não deve ser reproduzido sem a devida autorização do autor. Este material pode conter erros e imprecisões. O autor não se responsabiliza por quaisquer danos ou prejuízos decorrentes do uso deste material. O uso deste material é de responsabilidade exclusiva do usuário. Caso você encontre erros ou imprecisões neste material, por favor, entre em contato com o autor para que possam ser corrigidos. O autor agradece qualquer feedback ou sugestão de melhoria.*

## Conceitos Fundamentais Relacionados


#### **Validação Cruzada**

**Descrição:**  
A validação cruzada é uma técnica usada para avaliar a performance de um modelo dividindo os dados em partes de treinamento e teste, garantindo que o modelo seja avaliado de forma confiável e generalizável.

**Tipos de Validação Cruzada:**
- **K-Fold Cross-Validation:**  
  Divide os dados em $K$ partes (folds). Em cada iteração, um fold é usado como teste e os outros $K-1$ como treinamento.
  - **Vantagem:** Reduz a variância da avaliação do modelo.
- **Leave-One-Out Cross-Validation (LOOCV):**  
  Cada observação é usada como conjunto de teste uma vez, e todas as outras como treinamento.
  - **Vantagem:** Usa quase todos os dados para treinamento.
  - **Desvantagem:** Computacionalmente caro para grandes datasets.

**Importância:**
- Evita problemas de overfitting e underfitting.
- Proporciona uma estimativa mais confiável da performance do modelo.


#### **Overfitting e Underfitting**

**Overfitting:**  
Ocorre quando o modelo é excessivamente ajustado aos dados de treinamento, capturando ruídos e padrões específicos demais.  
- **Sintomas:** Alta performance no treinamento, mas baixa performance em novos dados.  
- **Exemplo:** Um modelo que ajusta exatamente cada ponto dos dados de treinamento.

**Underfitting:**  
Ocorre quando o modelo não consegue capturar a complexidade dos dados.  
- **Sintomas:** Baixa performance tanto no treinamento quanto no teste.  
- **Exemplo:** Um modelo linear usado para dados com uma relação não-linear.

**Visualização:**
- **Underfitting:** Modelo simples demais (reta em dados não-lineares).
- **Good Fit:** Modelo adequado para os padrões gerais.
- **Overfitting:** Modelo que segue cada ponto dos dados.


#### **Regularização**

**Descrição:**  
Regularização adiciona penalidades aos coeficientes do modelo para evitar que eles assumam valores extremos, reduzindo o risco de overfitting.

**Principais Técnicas:**
- **LASSO (Least Absolute Shrinkage and Selection Operator):**  
  Adiciona uma penalidade $\lambda \sum |\beta_j|$, promovendo coeficientes esparsos (alguns $\beta_j = 0$).
- **Ridge Regression:**  
  Adiciona uma penalidade $\lambda \sum \beta_j^2$, encolhendo os coeficientes, mas não os eliminando.
- **Elastic Net:**  
  Combinação de LASSO e Ridge, com penalidade $\lambda_1 \sum |\beta_j| + \lambda_2 \sum \beta_j^2$.

**Objetivo:**  
Controlar a complexidade do modelo para melhorar sua generalização.


#### **Feature Engineering**

**Descrição:**  
Feature engineering é o processo de criar, selecionar ou transformar variáveis (features) para melhorar a capacidade preditiva de um modelo.

**Passos Importantes:**
1. **Criação de Novas Features:**  
   Combinar variáveis existentes ou criar interações (ex.: multiplicação, logaritmo, transformações).
2. **Seleção de Features:**  
   Escolher as variáveis mais relevantes para reduzir dimensionalidade.
   - Técnicas comuns: Filtro, Wrapper, Embedded (ex.: LASSO).
3. **Transformação de Features:**  
   Padronizar ou normalizar variáveis numéricas para escalas uniformes.

**Impacto:**  
- Melhora a performance e interpretabilidade dos modelos.
- Reduz o risco de overfitting ao evitar features redundantes.


#### **Seleção de Modelos**

**Descrição:**  
Seleção de modelos é o processo de escolher o modelo que melhor equilibra performance e interpretabilidade.

**Critérios de Avaliação:**
- **$R^2$ Ajustado:** Para modelos de regressão.
- **AUC (Área sob a Curva ROC):** Para modelos de classificação.
- **RMSE (Root Mean Squared Error):** Para medir erros em predições contínuas.
- **Log Loss:** Para avaliar probabilidades preditivas em classificação.

**Trade-offs na Escolha:**
- **Complexidade:** Modelos mais complexos geralmente têm maior capacidade preditiva, mas podem ser difíceis de interpretar.
- **Interpretabilidade:** Modelos mais simples são mais fáceis de explicar, mas podem ter menor capacidade de capturar padrões complexos.



## O Uso de ML em Economia

Os problemas em economia frequentemente envolvem:

* **Previsão de Séries Temporais:** Previsão de variáveis macroeconômicas, como inflação, PIB e taxas de desemprego.
* **Análise de Impacto Causal:** Estimar o efeito de políticas públicas, como subsídios, impostos ou reformas.
* **Classificação e Segmentação:** Identificar padrões em dados de consumidores, mercados ou empresas.
* **Otimização de Decisões:** Melhorar políticas de alocação de recursos ou investimentos.

Técnicas Mais Comuns

* **Previsão:**
  * Modelos de séries temporais: ARIMA, Prophet, Redes Neurais Recorrentes (RNN) e LSTMs.
    * Algoritmos de regressão para prever preços, consumo, ou variáveis socioeconômicas.

* **Causalidade e Inferência:**
  * Double Machine Learning (DML), Modelos de Variáveis Instrumentais, Modelos Baseados em Controle Sintético.
    * Exemplo: Avaliar o impacto de uma política fiscal sobre o PIB.


## Considerações Finais

Machine Learning é uma ferramenta poderosa para analisar dados complexos e extrair insights valiosos em economia. Com a crescente disponibilidade de dados e o desenvolvimento de algoritmos sofisticados, o uso de ML na economia está se tornando cada vez mais comum.

O aprendizado supervisionado é predominante em economia. Isso ocorre por diversos fatores.
* Natureza dos Dados em Economia: Muitos estudos econômicos trabalham com dados rotulados, o que é ideal para aprendizado supervisionado.
* Foco em Predição e Causalidade: Economistas frequentemente estão interessados em prever uma variável dependente ou entender o impacto causal de uma política ou evento. Modelos supervisionados, como regressões, árvores de decisão, e redes neurais, são ferramentas diretas para esses objetivos.
* Interpretação e Explicabilidade: Modelos como regressões lineares, árvores de decisão e random forests são supervisionados e mais interpretáveis, uma característica valorizada em economia.

No entanto, técnicas de aprendizado não supervisionado, semi-supervisionado e por reforço também têm aplicações importantes em economia. Por exemplo, agrupamento de consumidores, redução de dimensionalidade em dados macroeconômicos e otimização de políticas públicas.